In [39]:
import pandas as pd
import numpy as np

In [3]:
help_full_path = './data/HELP/HELP_full.tsv'

In [4]:
# col_names = ['index', 'sentence1', 'sentence2', 'gold_label', 'gold_label_num']

In [77]:
from data_utils import load_help

In [78]:
train, test = load_help()
train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

2023-04-07 14:14:38.768 | INFO     | data_utils:load_help:37 - Index(['sentence1', 'sentence2', 'gold_label', 'gold_label_num'], dtype='object')
2023-04-07 14:14:38.815 | INFO     | data_utils:load_help:41 - (35891, 4)
Casting the dataset: 100%|██████████| 5/5 [00:00<00:00, 820.67ba/s]


In [79]:
help_original = pd.read_csv('data/HELP_original.tsv', sep='\t', index_col=0)
help_original['premise']  = help_original['ori_sentence']
help_original['hypothesis']  = help_original['new_sentence']

In [80]:
help_original

,filename,determiner,monotonicity,gold_label,replace_target,replace_source,replace_mode,ori_sentence,new_sentence,premise,hypothesis
0,silver/p75/d3454,neither,downward_monotone,entailment,neither one of his parents,neither parents,simple,Tom said that neither parents had ever been to...,Tom said that neither one of his parents had e...,Tom said that neither parents had ever been to...,Tom said that neither one of his parents had e...
1,silver/p14/d0959,neither,downward_monotone,entailment,neither of them,neither them,simple,I know neither them .,I know neither of them .,I know neither them .,I know neither of them .
2,silver/p24/d2663,neither,downward_monotone,entailment,neither of his brothers,neither brothers,simple,I know neither brothers .,I know neither of his brothers .,I know neither brothers .,I know neither of his brothers .
3,silver/p24/d2663,neither,downward_monotone,neutral,brothers,friends,noun_hypernym_obj,I know neither of his brothers .,I know neither of his friends .,I know neither of his brothers .,I know neither of his friends .
4,silver/p24/d2663,neither,downward_monotone,entailment,brothers,friends,noun_hypernym_obj,I know neither of his friends .,I know neither of his brothers .,I know neither of his friends .,I know neither of his brothers .
...,...,...,...,...,...,...,...,...,...,...,...
431,silver/p97/d2664,or,disjunction,entailment,NaN,NaN,NaN,There were too many sights to see in a day,There were too many sights to see in a day or ...,There were too many sights to see in a day,There were too many sights to see in a day or ...
432,silver/p97/d2794,or,disjunction,neutral,NaN,NaN,NaN,There are two or three pens on the desk .,There are two,There are two or three pens on the desk .,There are two
434,silver/p98/d2720,or,disjunction,entailment,NaN,NaN,NaN,Tom eats eggs without salt or pepper .,Tom eats eggs without salt,Tom eats eggs without salt or pepper .,Tom eats eggs without salt
435,silver/p98/d2720,or,disjunction,neutral,NaN,NaN,NaN,Tom eats eggs without salt,Tom eats eggs without salt or pepper .,Tom eats eggs without salt,Tom eats eggs without salt or pepper .


In [81]:
# join original labels to our help_test split

test_df = pd.merge(test_df, help_original, on=['premise', 'hypothesis'], how='left', indicator='Exist')

test_df['Exist'] = np.where(test_df.Exist=="both", True, False)
test_df = test_df.loc[test_df.Exist]




In [82]:

train_df = pd.merge(train_df, help_original, on=['premise', 'hypothesis'], how='left', indicator='Exist')

train_df['Exist'] = np.where(train_df.Exist=="both", True, False)
train_df = train_df.loc[test_df.Exist]




IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [86]:
train_df.monotonicity.value_counts()

downward_monotone    17909
upward_monotone       7046
conjunction           6044
non_monotone           992
disjunction            342
Name: monotonicity, dtype: int64

In [140]:
def extract_context(row):
    # the replace target/source is not consistent with premise/hypothesis, so we try both and take the successful one:
    premise = row['premise']
    context = premise.replace(row['replace_source'] ,'x')
    check_context = premise.replace(row['replace_target'] ,'x')

    if context != premise:
        return context
    elif check_context != premise:
        return check_context
    else: 
        print(context)
        print(replace_source)
        print(check_context)
        print(replace_target)
        raise ValueError


In [141]:
sample_for_upward_contexts = train_df.loc[train_df.monotonicity=='upward_monotone'].sample(1000)
upward_contexts = sample_for_upward_contexts.apply(extract_context, axis=1)
upward_contexts


ValueError: 

In [119]:
sample_for_downward_contexts = train_df.loc[train_df.monotonicity=='downward_monotone'].sample(1000)
downward_contexts = sample_for_downward_contexts.apply(extract_context, axis=1)
downward_contexts

Tom did n't reply to any of Mary 's w .
letter
Tom did n't reply to any of Mary 's w .
Tom did n't reply to any of Mary 's letter .
He was known to every romper in the village .
body
He was known to every romper in the village .
He was known to every body in the village .
There is no beach in this cleavage .
area
There is no beach in this cleavage .
There is no beach in this area .
Taiwan was far from being any kind of economic miracle in the decennaries .
1950s
Taiwan was far from being any kind of economic miracle in the decennaries .
Taiwan was far from being any kind of economic miracle in the 1950s .
When Tom went to have breakfast , he found that all bread had gone mouldy .
all the bread
When Tom went to have breakfast , he found that all bread had gone mouldy .
When Tom went to have breakfast , he found that all the bread had gone mouldy .
He ate it in no meter reading .
time
He ate it in no meter reading .
He ate it in no time .
My dog barks all the musical time .
time
My dog b

16648              Tom did n't reply to any of Mary 's w .
17328    At the time there were no native English speak...
11909             He was known to every x in the village .
11718        He was known to every romper in the village .
9605                  There is no beach in this cleavage .
                               ...                        
14134                             My dog barks all the x .
2138                          You have no reply to shins .
1272                                     Tom has no x on .
13592                         I ate every thing on the x .
16241            Tom plans to put new locks on all the x .
Length: 1000, dtype: object

In [106]:
with open('data/HELP_Contexts/sample_for_upward_contexts.tsv', 'w+') as file:
    file.write(sample_for_upward_contexts.to_csv(sep='\t'))